In [3]:
# Import dependencies
import requests
from config import spotify_token
import pandas as pd
import numpy as np
import time
import re

In [11]:
# Create DataFrame from Spotify API calls
song_genres_df = pd.read_csv('../../Data/song_genres.csv')
song_genres_df = song_genres_df.dropna()
song_genres_df.head(3)

,song,song_id,artist,artist_id,category,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.00179,0.145,0.529,81.112,214707.0,4.0
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.00000,0.366,0.756,179.954,206046.0,4.0
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.00000,0.134,0.467,93.005,161385.0,4.0


In [16]:
base_url = 'https://api.spotify.com'
headers_dict = {'Content-Type': 'application/json', 
                'Authorization': f'Bearer {spotify_token}',
                'country': 'US'}

In [13]:
# Create a list of filtered categories
filtered_categories = ['pop', 'hiphop', 'metal', 'rock', 'jazz', 'rnb',
                       'romance', 'soul', 'indie_alt', 'classical', 'blues', 'funk',
                       'punk', 'country']
other = ['edm_dance', 'party', 'family', 'caribbean', 'afro']

In [25]:
# Add a popularity column
popularities = []
for category in filtered_categories:
    t0 = time.time()
    #filtered_df = song_genres_df[:].copy()
    #filtered_df = filtered_df[filtered_df['category']==category]
    filtered_df = song_genres_df[song_genres_df['category']==category].copy()
    for index, row in filtered_df.iterrows():
        song_id = row['song_id']
        # Get track information
        query_url = base_url + f'/v1/tracks/{song_id}'
        results = requests.get(query_url, headers=headers_dict).json()
        popularities.append(results['popularity'])
    t1 = time.time()
    print(f'Run time to add {len(filtered_df)} {category} song popularities: {t1-t0}')
song_genres_df.insert(5, 'popularity', popularities)
song_genres_df.head(3)

Run time to add 855 pop song popularities: 112.20797324180603
Run time to add 881 hiphop song popularities: 117.11923027038574
Run time to add 1259 metal song popularities: 165.77524399757385
Run time to add 1625 rock song popularities: 214.1269919872284
Run time to add 1298 jazz song popularities: 169.20270586013794
Run time to add 404 rnb song popularities: 51.6838960647583
Run time to add 357 romance song popularities: 45.404722929000854
Run time to add 569 soul song popularities: 73.40592908859253
Run time to add 813 indie_alt song popularities: 105.3696551322937
Run time to add 2203 classical song popularities: 286.5459761619568
Run time to add 925 blues song popularities: 116.62349796295166
Run time to add 750 funk song popularities: 97.78203296661377
Run time to add 811 punk song popularities: 105.05749297142029
Run time to add 1461 country song popularities: 233.1321520805359


,song,song_id,artist,artist_id,category,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.00179,0.145,0.529,81.112,214707.0,4.0
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.00000,0.366,0.756,179.954,206046.0,4.0
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.00000,0.134,0.467,93.005,161385.0,4.0


In [26]:
# Save DataFrame to CSV
song_genres_df.to_csv('../../Data/song_genres_popularity.csv', index=False)